In [ ]:
!wget -c http://gemeos.org/henrique/health_fasttext_300v1.tar.gz -O health_fasttext_300v1.tar.gz #0.86
!tar -zxvf health_fasttext_300v1.tar.gz

--2020-02-12 16:57:28--  http://gemeos.org/henrique/health_fasttext_300v1.tar.gz
Resolving gemeos.org (gemeos.org)... 187.45.195.133
Connecting to gemeos.org (gemeos.org)|187.45.195.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2255702828 (2.1G) [application/x-gzip]
Saving to: ‘health_fasttext_300v1.tar.gz’

health_fasttext_300 100%[===================>]   2.10G  24.3MB/s    in 90s     

2020-02-12 16:58:58 (23.9 MB/s) - ‘health_fasttext_300v1.tar.gz’ saved [2255702828/2255702828]

health_fasttext_300.model
health_fasttext_300.model.trainables.syn1neg.npy
health_fasttext_300.model.trainables.vectors_ngrams_lockf.npy
health_fasttext_300.model.trainables.vectors_vocab_lockf.npy
health_fasttext_300.model.wv.vectors_ngrams.npy
health_fasttext_300.model.wv.vectors.npy
health_fasttext_300.model.wv.vectors_vocab.npy


In [ ]:
!pip install flair==0.4.3

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.embeddings import DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.optim import SGDW
from flair.data import Sentence
from sklearn.model_selection import StratifiedKFold
from typing import List
import pandas as pd
import torch
import os

## FOLD 1

In [ ]:
pathCorpus : str = '/content/Fall-Recognition/Data/FOLDER_1/'
train_file : str = 'Train-SPLIT-1.txt'
test_file : str = 'Test-SPLIT-1.txt'
dev_file : str = 'CoNLL-dev.txt'
pathCheckpoint: str = '/content/Fall-Recognition/Data/FOLDER_1/TRN-FD1'
pathWordEmbeddings : str = 'health_fasttext_300.model'
pathFlairEmbeddingsForward : str = None
pathFlairEmbeddingsBackward : str =  None

print('--------------------------START TRAINING (TOKEN)------------------------------')

columns = {0:'token', 1:'label'}

corpus: Corpus = ColumnCorpus(pathCorpus, columns,
	train_file = train_file,
	test_file = test_file,
	dev_file = dev_file)

print(" ")
print("Train len: ", len(corpus.train))
print("Test len: ", len(corpus.test))
print("Dev len: ", len(corpus.dev))

tag_type = 'label'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

print(tag_dictionary.idx2item)
print("len: ", len(tag_dictionary.idx2item))

health_embedding = WordEmbeddings(pathWordEmbeddings)
#flair_forward_embedding = FlairEmbeddings(pathFlairEmbeddingsForward)
#flair_backward_embedding = FlairEmbeddings(pathFlairEmbeddingsBackward)

embedding_types: List[TokenEmbeddings] = [
	health_embedding,
]
  
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
	
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
										embeddings=embeddings,
										tag_dictionary=tag_dictionary,
										tag_type=tag_type,
										use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=SGDW)

trainer.train(pathCheckpoint,
			  learning_rate=0.1,
			  mini_batch_size=32,
			  max_epochs=150,
			  checkpoint=True)

--------------------------START TRAINING (TOKEN)------------------------------
2020-02-12 12:03:31,277 Reading data from /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_1
2020-02-12 12:03:31,280 Train: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_1/Train-SPLIT-1.txt
2020-02-12 12:03:31,281 Dev: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_1/CoNLL-dev.txt
2020-02-12 12:03:31,283 Test: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_1/Test-SPLIT-1.txt
 
Train len:  888
Test len:  444
Dev len:  70
[b'<unk>', b'O', b'B-QUEDA', b'I-QUEDA', b'<START>', b'<STOP>']
len:  6


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-02-12 12:03:49,736 ----------------------------------------------------------------------------------------------------
2020-02-12 12:03:49,737 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('health_fasttext_300.model')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
)"
2020-02-12 12:03:49,739 ----------------------------------------------------------------------------------------------------
2020-02-12 12:03:49,740 Corpus: "Corpus: 888 train + 70 dev + 444 test sentences"
2020-02-12 12:03:49,742 ----------------------------------------------------------------------------------------------------
2020-02-12 12:03:49,744 Parameters:
2020-02-12 12:03:49,745  - learning_rate: "0.1"
2020-02-12 12:03:49,746  - mini_batch_size: 

{'test_score': 0.5972,
 'dev_score_history': [0.4913,
  0.6046,
  0.5417,
  0.5957,
  0.6383,
  0.6364,
  0.5957,
  0.5652,
  0.6364,
  0.6087,
  0.6667,
  0.6087,
  0.625,
  0.6,
  0.6522,
  0.6522,
  0.6667,
  0.6522,
  0.5715,
  0.6667,
  0.6383,
  0.625,
  0.625,
  0.625,
  0.6383,
  0.6383,
  0.625,
  0.625,
  0.625,
  0.625,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383,
  0.6383],
 'train_loss_history': [31.945099455969675,
  2.8073516147477284,
  2.325678672109331,
  2.0447819318090166,
  1.8833965403693063,
  1.6827706694602966,
  1.5696230615888322,
  1.4505006628377097,
  1.3877758809498377,
  1.299524085862296,
  1.2179920162473405,
  1.2768209746905737,
  1.1658373858247484,
  1.2321813787732805,
  1.2051009365490504,
  1.1651398880141122,
  1.144176324563367,
  1.0472690165042877,
  1.029554226568767,
  1.0403904403959001,
  1.0619499768529619,
  1.040403093610

## FOLD 2

In [ ]:
pathCorpus : str = '/content/Fall-Recognition/Data/FOLDER_2/'
train_file : str = 'Train-SPLIT-2.txt'
test_file : str = 'Test-SPLIT-2.txt'
dev_file : str = 'CoNLL-dev.txt'
pathCheckpoint: str = '/content/Fall-Recognition/Data/FOLDER_2/TRN-FD2'
pathWordEmbeddings : str = 'health_fasttext_300.model'
pathFlairEmbeddingsForward : str = None
pathFlairEmbeddingsBackward : str =  None

print('--------------------------START TRAINING (TOKEN)------------------------------')

columns = {0:'token', 1:'label'}

corpus: Corpus = ColumnCorpus(pathCorpus, columns,
	train_file = train_file,
	test_file = test_file,
	dev_file = dev_file)

print(" ")
print("Train len: ", len(corpus.train))
print("Test len: ", len(corpus.test))
print("Dev len: ", len(corpus.dev))

tag_type = 'label'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

print(tag_dictionary.idx2item)
print("len: ", len(tag_dictionary.idx2item))

health_embedding = WordEmbeddings(pathWordEmbeddings)
#flair_forward_embedding = FlairEmbeddings(pathFlairEmbeddingsForward)
#flair_backward_embedding = FlairEmbeddings(pathFlairEmbeddingsBackward)

embedding_types: List[TokenEmbeddings] = [
	health_embedding,
]
  
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
	
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
										embeddings=embeddings,
										tag_dictionary=tag_dictionary,
										tag_type=tag_type,
										use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=SGDW)

trainer.train(pathCheckpoint,
			  learning_rate=0.1,
			  mini_batch_size=32,
			  max_epochs=150,
			  checkpoint=True)

--------------------------START TRAINING (TOKEN)------------------------------
2020-02-12 14:29:19,376 Reading data from /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_2
2020-02-12 14:29:19,380 Train: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_2/Train-SPLIT-2.txt
2020-02-12 14:29:19,381 Dev: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_2/CoNLL-dev.txt
2020-02-12 14:29:19,382 Test: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_2/Test-SPLIT-2.txt
 
Train len:  888
Test len:  444
Dev len:  70
[b'<unk>', b'O', b'B-QUEDA', b'I-QUEDA', b'<START>', b'<STOP>']
len:  6


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-02-12 14:31:28,557 ----------------------------------------------------------------------------------------------------
2020-02-12 14:31:28,559 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('health_fasttext_300.model')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
)"
2020-02-12 14:31:28,560 ----------------------------------------------------------------------------------------------------
2020-02-12 14:31:28,563 Corpus: "Corpus: 888 train + 70 dev + 444 test sentences"
2020-02-12 14:31:28,564 ----------------------------------------------------------------------------------------------------
2020-02-12 14:31:28,567 Parameters:
2020-02-12 14:31:28,568  - learning_rate: "0.1"
2020-02-12 14:31:28,569  - mini_batch_size: 

{'test_score': 0.5487,
 'dev_score_history': [0.4,
  0.3824,
  0.619,
  0.6809,
  0.7111,
  0.625,
  0.6809,
  0.6829,
  0.6977,
  0.6818,
  0.6809,
  0.6957,
  0.6809,
  0.6122,
  0.6122,
  0.6809,
  0.6531,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667,
  0.6667],
 'train_loss_history': [31.719597629138402,
  3.506468823977879,
  2.929951557091304,
  2.557249801499503,
  2.2359372547694614,
  2.0809710877282277,
  1.8982645188059126,
  1.7724333831242152,
  1.7003732919692993,
  1.5558743902615138,
  1.502244621515274,
  1.4588197299412318,
  1.376197725534439,
  1.3836767737354552,
  1.292341330221721,
  1.3230521636349815,
  1.286066702433995,
  1.2599465123244695,
  1.184630913393838,
  1.2242426020758492,
  1.1742317931992667,
  1.1576455278056008,
  1.21859687

## FOLD 3

In [ ]:
pathCorpus : str = '/content/Fall-Recognition/Data/FOLDER_3/'
train_file : str = 'Train-SPLIT-3.txt'
test_file : str = 'Test-SPLIT-3.txt'
dev_file : str = 'CoNLL-dev.txt'
pathCheckpoint: str = '/content/Fall-Recognition/Data/FOLDER_3/TRN-FD3'
pathWordEmbeddings : str = 'health_fasttext_300.model'
pathFlairEmbeddingsForward : str = None
pathFlairEmbeddingsBackward : str =  None

print('--------------------------START TRAINING (TOKEN)------------------------------')

columns = {0:'token', 1:'label'}

corpus: Corpus = ColumnCorpus(pathCorpus, columns,
	train_file = train_file,
	test_file = test_file,
	dev_file = dev_file)

print(" ")
print("Train len: ", len(corpus.train))
print("Test len: ", len(corpus.test))
print("Dev len: ", len(corpus.dev))

tag_type = 'label'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

print(tag_dictionary.idx2item)
print("len: ", len(tag_dictionary.idx2item))

health_embedding = WordEmbeddings(pathWordEmbeddings)
#flair_forward_embedding = FlairEmbeddings(pathFlairEmbeddingsForward)
#flair_backward_embedding = FlairEmbeddings(pathFlairEmbeddingsBackward)

embedding_types: List[TokenEmbeddings] = [
	health_embedding,
]
  
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
	
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
										embeddings=embeddings,
										tag_dictionary=tag_dictionary,
										tag_type=tag_type,
										use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=SGDW)

trainer.train(pathCheckpoint,
			  learning_rate=0.1,
			  mini_batch_size=32,
			  max_epochs=150,
			  checkpoint=True)

--------------------------START TRAINING (TOKEN)------------------------------
2020-02-12 17:03:01,509 Reading data from /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_3
2020-02-12 17:03:01,510 Train: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_3/Train-SPLIT-3.txt
2020-02-12 17:03:01,511 Dev: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_3/CoNLL-dev.txt
2020-02-12 17:03:01,512 Test: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/FOLDER_3/Test-SPLIT-3.txt
 
Train len:  888
Test len:  444
Dev len:  70
[b'<unk>', b'O', b'B-QUEDA', b'I-QUEDA', b'<START>', b'<STOP>']
len:  6


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-02-12 17:03:23,056 ----------------------------------------------------------------------------------------------------
2020-02-12 17:03:23,058 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('health_fasttext_300.model')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
)"
2020-02-12 17:03:23,059 ----------------------------------------------------------------------------------------------------
2020-02-12 17:03:23,061 Corpus: "Corpus: 888 train + 70 dev + 444 test sentences"
2020-02-12 17:03:23,062 ----------------------------------------------------------------------------------------------------
2020-02-12 17:03:23,064 Parameters:
2020-02-12 17:03:23,066  - learning_rate: "0.1"
2020-02-12 17:03:23,067  - mini_batch_size: 

{'test_score': 0.5988,
 'dev_score_history': [0.4783,
  0.6383,
  0.6809,
  0.6522,
  0.579,
  0.4444,
  0.6809,
  0.6667,
  0.6809,
  0.6122,
  0.6809,
  0.6667,
  0.6531,
  0.6531,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64,
  0.64],
 'train_loss_history': [24.54027901376997,
  3.385047197341919,
  2.6585456005164554,
  2.3565533586910794,
  2.138084718159267,
  1.8553393823759896,
  1.7396685906818934,
  1.6574717632361822,
  1.5599483847618103,
  1.4593707195350103,
  1.4836481426443373,
  1.3921161677156175,
  1.3943055995873042,
  1.3340523838996887,
  1.304547735622951,
  1.2546399299587523,
  1.2916852704116277,
  1.2640106465135301,
  1.2748524631772722,
  1.2146339203630174,
  1.2102458934698785,
  1.283508654151644,
  1.3110862629754203,
  1.234634348324367,
  1.2234178334474564,
  1.19619147692407

## Evaluating

In [ ]:
!python3 EvaluatesModels.py

--------------- (NER) EVAL CROSS-VALIDATION FOLDER: 1 ---------------
 
--------------------------START VALIDATION TOKEN------------------------------
 
2020-02-14 10:56:09,670 loading file /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Experiments/CROSS-NER/FOLDER_1/TRN-FD1/best-model.pt
 
 --- VALID --- 
              precision    recall  f1-score   support

     class 0       0.95      0.96      0.96      1913
     class 1       0.85      0.79      0.82       477

    accuracy                           0.93      2390
   macro avg       0.90      0.88      0.89      2390
weighted avg       0.93      0.93      0.93      2390

	F1 Score

Class 0	 0.9562062710546774
Class 1	 0.8165038002171552
 
--------------- (NER) EVAL CROSS-VALIDATION FOLDER: 2 ---------------
 
--------------------------START VALIDATION TOKEN------------------------------
 
2020-02-14 10:59:55,680 loading file /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Experiments/CROSS-NER/FOLDER_2/TRN-FD2/